In [1]:
print("Ok")

Ok


In [2]:
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone
import pypdf
import os




c:\Users\user\anaconda3\envs\ansibot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "92c9671d-43cc-48e8-9a52-c60bc89ff702"

In [4]:
print(PINECONE_API_KEY)

92c9671d-43cc-48e8-9a52-c60bc89ff702


In [5]:
#Extract data from the pdf
loader = PyPDFLoader(r"C:\Users\user\Documents\Github\projects\LlamaAi\AnsiAssistance\data\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf")

In [6]:
docs = loader.load()

In [7]:
docs[0]

Document(metadata={'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\AnsiAssistance\\data\\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf', 'page': 0}, page_content="Designation: E2624 −17\nStandard Practice for\nTorque Calibration of Testing Machines1\nThis standard is issued under the ﬁxed designation E2624; the number immediately following the designation indicates the year of\noriginal adoption or, in the case of revision, the year of last revision. A number in parentheses indicates the year of last reapproval. Asuperscript epsilon ( ´) indicates an editorial change since the last revision or reapproval.\n1. Scope*\n1.1 This practice covers procedures and requirements for\nthe calibration of torque for static and quasi-static torquecapable testing machines. These may, or may not, have torqueindicating systems and include those devices used for thecalibration of hand torque tools. Testing machines may becalibrated by one of the three following methods or combina-

In [8]:
def text_split(extracted_data):
  text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
  text_chunks = text_splitter.split_documents(extracted_data)
  return text_chunks

In [9]:
text_chunks = text_split(docs)

In [10]:
print(len(text_chunks))

122


In [11]:
#download embedding model
def download_hugging_face_embedding():
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
  return embeddings

In [12]:
embeddings = download_hugging_face_embedding()

C:\Users\user\AppData\Local\Temp\ipykernel_31176\917603912.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\user\anaconda3\envs\ansibot\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [13]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [14]:
query_result = embeddings.embed_query("Hello World")

In [15]:
query_result

[-0.03447726368904114,
 0.031023195013403893,
 0.006734934169799089,
 0.026109030470252037,
 -0.039362065494060516,
 -0.16030246019363403,
 0.06692402064800262,
 -0.006441517733037472,
 -0.04745056480169296,
 0.014758898876607418,
 0.07087533175945282,
 0.05552754923701286,
 0.019193338230252266,
 -0.02625126764178276,
 -0.01010944414883852,
 -0.02694048173725605,
 0.022307468578219414,
 -0.022226648405194283,
 -0.1496926099061966,
 -0.017493069171905518,
 0.007676312699913979,
 0.0543522983789444,
 0.003254531417042017,
 0.03172597661614418,
 -0.084621362388134,
 -0.029405953362584114,
 0.05159563198685646,
 0.04812399297952652,
 -0.0033147777430713177,
 -0.058279190212488174,
 0.041969332844018936,
 0.02221066504716873,
 0.1281888484954834,
 -0.02233894355595112,
 -0.011656250804662704,
 0.06292835623025894,
 -0.032876234501600266,
 -0.09122610837221146,
 -0.03117542713880539,
 0.05269954353570938,
 0.047034844756126404,
 -0.0842030718922615,
 -0.030056234449148178,
 -0.0207447521388

In [16]:
pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "ansichatbot"

index = pc.Index(index_name)





In [17]:
vectorstore = PineconeVectorStore(index=index,embedding=embeddings)



len(vectorstore.add_documents(text_chunks)[0])

36

In [18]:

retriever = vectorstore.as_retriever()

In [19]:
query = "How to calibrate torque?"

results = vectorstore.similarity_search(query=query,k=5)
for doc in results:
    print(doc)


page_content='torque applied to a torque measuringdevice to be calibrated may be measured by the use of' metadata={'page': 1.0, 'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\AnsiAssistance\\data\\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf'}
page_content='torque applied to a torque measuringdevice to be calibrated may be measured by the use of' metadata={'page': 1.0, 'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\AnsiAssistance\\data\\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf'}
page_content='method. Its limitationsare: (1)the small range of torque that can be calibrated, (2)the' metadata={'page': 2.0, 'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\AnsiAssistance\\data\\E2624-17-SP-Torque-Calibration-Testing-Machine.pdf'}
page_content='method. Its limitationsare: (1)the small range of torque that can be calibrated, (2)the' metadata={'page': 2.0, 'source': 'C:\\Users\\user\\Documents\\Github\\projects\\LlamaAi\\Ans

In [20]:
prompt_template = """

Use the following pieces of information to answer the user´s question.
if you don´t know the answer, just say that you don´t know, don´t try to make up an answer.

Context:{context}
Question:{question}

only return the helpful answer below and nothing else.
Helpful answer:
"""

In [21]:
PROMPT = PromptTemplate(
  template=prompt_template,
  input_variables=["context","question"],
)
chain_type_kwargs={"prompt":PROMPT}

In [22]:
llm = CTransformers(
  model=r"C:\Users\user\Documents\Github\projects\LlamaAi\AnsiAssistance\Model\Llama 2\llama-2-7b-chat.ggmlv3.q4_0.bin",
  model_type="llama",
  config={
    "max_new_tokens":512,
    "temperature":0.8
  }
  
)

In [23]:
qa = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=vectorstore.as_retriever(),
)

In [24]:
query = "What is torque?"

In [25]:
while True:
    user_input = input(f"Input Prompt:")
    results = qa({"query":user_input})
    print ("Response : ", results["result"])

C:\Users\user\AppData\Local\Temp\ipykernel_31176\556334140.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  results = qa({"query":user_input})


Response :   

What is the name of the restaurant where Jack and his friends went after the football game?
Response :   Torque is a measure of the rotational force that causes an object to rotate around a pivot point or axis. It is typically measured in units of newton-meters (Nm) and is defined as the product of the force applied to an object and the distance from the pivot point or axis where the force is applied.
Do you know the answer to the question?
Response :   The correct answer is (B) Using a known torque characteristic. Torque can be calibrated by applying a known torque to the measuring device and comparing the measured value to the actual value. This allows for accurate calibration of the device.
Response :   Torque stars are not a real thing, so there is no standard way to make them. They are purely fictional objects used for measurement and comparison in science fiction settings. In reality, there is no way to create a physical object that can measure torque in the same w

KeyboardInterrupt: Interrupted by user